## Tâche 177 - Créer un modele DBT pour les résultats PFAS

### Contexte :

Partir de `select * from int__resultats_udi_communes` (cf https://data-for-good.slack.com/archives/C08B329AG7M/p1742063272779779)

Nous voulons calculer les résultats/situations pour les PFAS pour le dernier relevé par UDI (Prio 1).

6 situations:

- Pas recherché
- Aucun paramètre n’a été quantifié
- Au moins 1 paramètre a été quantifié mais la somme des 20 PFAS < 0,1 µg/L et la somme des 4 PFAS < 0,02 µg/L
- Somme des 20 PFAS < 0,1 µg/L et la somme des 4 PFAS (PFOA, PFOS, PFNA, PFHXS) > 0.02 µg/L
- Somme des 20 PFAS > 0,1 µg/L (fait passer l’affichage total polluant en orange)
- Au moins 1 PFAS > valeur sanitaire (fait passer l’affichage total polluant en rouge)

(pour le bilan annuel, aller lire le brief)

### Choses à faire :

Créer un modèle dbt pour "derniers resultat pfas udi"

Ecrire le code SQL qui produit le résultat table avec ces colonnes :

- cdreseau (dans le cas d'UDI, sinon inseecommune)
- periode ("dernier relevé" ou l'année)
- resultat : le résultat / la situation
- date : quand c'est le dernier relévé, rajouter le datetimeprel

Trouver un exemple de commune dans chaque cas pour illustrer avec les prélèvements bruts associés

---


### Exploration et familiarisation avec la table


In [1]:
%reload_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 20

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [3]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE categorie = 'pfas'
LIMIT 5

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00700187109,PFBA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFBS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00


In [4]:
%%sql

SELECT DISTINCT libmajparametre, cdparametre, cdparametresiseeaux
FROM stg_edc__resultats
WHERE libmajparametre LIKE 'SOMME DE % PFAS'
GROUP BY 1, 2, 3

Running query in 'duckdb:///../../database/data.duckdb'

libmajparametre,cdparametre,cdparametresiseeaux
SOMME DE 20 PFAS,8847,SPFAS


Objectif de la query du dessus : retrouver les codes et noms des parametres "Somme des 20 PFAS" et "Somme des 4 PFAS".

Résultat : il n'existe qu'un parametre "Somme des 20 PFAS". Il faudra alors créer artificiellemet une colonne "Somme des 4 PFAS".


In [5]:
%%sql

SELECT DISTINCT cdparametre, cdparametresiseeaux
FROM stg_edc__resultats
WHERE cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
GROUP BY 1, 2

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdparametre,cdparametresiseeaux
5347,PFOA
6830,PFHXS
6508,PFNA
6561,PFOS


In [6]:
%%sql

WITH latest_pfas_results AS (
    SELECT
        *,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            ) AS row_number
    FROM int__resultats_udi_communes
    WHERE categorie = 'pfas'
)

SELECT 
    referenceprel, 
    cdreseau, 
    datetimeprel,
    SUM(CASE WHEN cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS') THEN valtraduite ELSE 0 END) AS somme_4_pfas
FROM latest_pfas_results
WHERE row_number = 1
GROUP BY referenceprel, cdreseau, datetimeprel
ORDER BY somme_4_pfas DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,datetimeprel,somme_4_pfas
02A00113033,02A000841,2023-04-24 09:45:00,10.000
01300273898,013001457,2025-01-21 09:40:00,0.396
03400327431,034000985,2025-01-15 14:24:00,0.112
01100172447,011000329,2025-01-24 11:05:00,0.097
01100172447,011004114,2025-01-24 11:05:00,0.097
01300273795,013000416,2025-01-21 08:48:00,0.085
03400327430,034001188,2025-01-13 14:44:00,0.064
08900130113,089000574,2024-03-22 10:47:00,0.061
03900122750,039001435,2024-09-04 09:28:00,0.058
01800095087,018000371,2024-11-14 10:21:00,0.057


Ci-dessus : les prélèvements qui ont les plus grosses valeurs pour "Somme des 4 PFAS". On note que le prélèvement `02A00113033` a une valeur de 10. Peut-être une erreur (fort probable).


In [7]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE referenceprel = '02A00113033' AND cdreseau = '02A000841' AND categorie = 'pfas'

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
02A00113033,PFNA,10.000,None,2023,None,None,pfas,02A000841,2A276,2023-04-24 09:45:00


Ci-dessus : exploration des résultats pour la catégorie 'pfas' pour le prélèvement étrange de l'étape d'avant (`02A00113033`)

---


### Prio 1 : Resultats PFAS - UDI - Derniers Prélèvements


In [25]:
%%sql

--save last_results_pfas_udi

WITH latest_pfas_results AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, cdparametresiseeaux
            ORDER BY datetimeprel DESC
        ) AS row_number
    FROM int__resultats_udi_communes
    WHERE categorie = 'pfas'
    AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

aggregated_results AS (
    SELECT
        referenceprel,
        cdreseau,
        datetimeprel,
        MAX(CASE WHEN cdparametresiseeaux = 'SPFAS' THEN valtraduite ELSE 0 END) AS sum_20_pfas,
        SUM(CASE WHEN cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS') THEN valtraduite ELSE 0 END) AS sum_4_pfas,
        MAX(CASE WHEN cdparametresiseeaux != 'SPFAS' THEN valtraduite ELSE NULL END) AS max_individual_pfas,
        COUNT(DISTINCT CASE WHEN valtraduite != 0 THEN cdparametresiseeaux END) AS nb_quantified_params
    FROM latest_pfas_results
    WHERE row_number = 1
    GROUP BY referenceprel, cdreseau, datetimeprel
)

SELECT
    referenceprel,
    cdreseau,
    'pfas' AS categorie,
    datetimeprel AS last_datetimeprel,
    CASE
        WHEN sum_20_pfas IS NULL AND nb_quantified_params = 0 THEN 'Aucun paramètre n’a été quantifié'
        WHEN sum_20_pfas < 0.1 AND sum_4_pfas < 0.02 THEN 'Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L'
        WHEN sum_20_pfas < 0.1 AND sum_4_pfas >= 0.02 THEN 'Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS >= 0,02 µg/L'
        WHEN sum_20_pfas >= 0.1 THEN 'Somme des 20 PFAS >= 0,1 µg/L'
        WHEN max_individual_pfas >= 0.1 THEN 'Au moins 1 PFAS >= valeur sanitaire (0,1 µg/L)'
        ELSE 'Erreur de classification'
    END AS result,
    'dernier prélèvment' AS periode
FROM aggregated_results
ORDER BY datetimeprel DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,categorie,last_datetimeprel,result,periode
02600171893,026001154,pfas,2025-01-31 12:23:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
00500134428,005001232,pfas,2025-01-31 11:17:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04500164331,045000430,pfas,2025-01-31 10:02:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04500164331,045000431,pfas,2025-01-31 10:02:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04500164334,045000494,pfas,2025-01-31 09:38:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
03800342072,038000488,pfas,2025-01-31 09:15:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04400263448,044000652,pfas,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04400263448,044003468,pfas,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04400263448,044000151,pfas,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment
04400263448,044000155,pfas,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L",dernier prélèvment


### CHECKS : Resultats PFAS - UDI - Derniers Prélèvements


In [ ]:
%%sql

--with last_results_pfas_udi

SELECT result, MIN(referenceprel) AS referenceprel_test, MIN(cdreseau) AS cdreseau_test, COUNT(DISTINCT referenceprel) AS nb_prel_unique
FROM last_results_pfas_udi
GROUP BY result
ORDER BY result DESC

In [93]:
%%sql

WITH latest_pfas_results AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, cdparametresiseeaux
            ORDER BY datetimeprel DESC
        ) AS row_number
    FROM int__resultats_udi_communes
    WHERE categorie = 'pfas'
)

SELECT COUNT(DISTINCT referenceprel) AS nb_prel_unique
FROM latest_pfas_results
WHERE row_number = 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

nb_prel_unique
1164


In [87]:
%%sql --with last_results_pfas_udi

SELECT *
FROM last_results_pfas_udi
WHERE result = 'Erreur de classification'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,datetimeprel,result
07200133081,072000498,2023-04-14 10:39:00,Erreur de classification
07200133081,072000924,2023-04-14 10:39:00,Erreur de classification


In [94]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE referenceprel = '07200133081' AND cdreseau = '072000498' AND categorie = 'pfas'
ORDER BY inseecommune ASC

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72012,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72152,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72297,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72332,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72362,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72370,2023-04-14 10:39:00


Que faire dans le cas ci-dessus ? Edge case : valtraduite IS NULL --> que faire ?
